# PACKAGES

In [ ]:
!pip install senticnet #for sentiment analysis (takes concept)
!pip install urllib3==1.26.15 requests-toolbelt==0.10.1
!pip install nervaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [ ]:
!pip install inceptalytics==0.1.0

In [ ]:
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.Defaults.stop_words |= {"chapter","title", "author", "date"} #add corpus-specific stopwords

from senticnet.senticnet import SenticNet
sn = SenticNet() #English
from senticnet.babelsenticnet import BabelSenticNet #= multilingual
bsn = BabelSenticNet('it') #Italian

import statistics
import numpy as np

from nervaluate import Evaluator
#from inceptalytics import Project

from spacy.matcher import Matcher
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
import string
from sklearn.metrics import classification_report

[nltk_data] Downloading package wordnet to /root/nltk_data...


# DATA
* Load data



In [ ]:
df = pd.read_csv("/content/drive/MyDrive/1. WORK IN PROGRESS/PhD/ABSA notebooks/IT_travel_all.csv")

NameError: name 'pd' is not defined

In [ ]:
df.head()

In [ ]:
len(df)

# RAW PROTOTYPE


1.  Prepare data (lemmatization and stop word removal)
2.  POS-tagging (extract noun phrases and relevant constructions) for aspect extraction
  *  Which constructions to take into account?
  Based on: Aspect Extraction Using Coreference Resolution and Unsupervised Filtering 2020 ;
  https://www.enjoyalgorithms.com/blog/aspect-base-sentiment-analysis-in-python (has a paper) ;
  https://www.researchgate.net/publication/370416967_Aspect-based_Sentiment_Analysis_on_Car_Reviews_Using_SpaCy_Dependency_Parsing_and_VADER
3.  Apply sentiment dictionary.

--> Choose sentiment approach based on dichotomy in https://www.digitalhumanities.org/dhq/vol/17/2/000691/000691.html#elkins2022 .
Paper: it is okay to use lexica when training data is limited.

Cater to:
*   Concept drift (ensure scores are somehow adapted to the sentiment of the word in the past)
  * Paper: study concept drift in 150 year English literature (methodology 2019)
  * Paper: Simple, Interpretable and Stable Method
for Detecting Words with Usage Change across Corpora
*   Multilinguality (ensure aspects are extracted in both languages Italian and English + sentiment of words are catered to)
  * Strategy? Translate into English?
*   Evaluate strategy while taking these things into account vs. not taking them into account.


4.  Evaluate extracted aspects and sentiment.

*   Assume the gold standard includes the "true sentiment" of the sentences. It doesn't really matter how you extract it in the methodology. Issue though: the gold standard data does not always include Italian words. Should we then go for a multilingual strategy (irrespective of language? Not necessarily tailored to "Italian").
*  Evaluate on IT pieces? Only 3 - not super representative perhaps


In [ ]:
test_text = df.iloc[0]['text']

#EXTRACT INFORMATION: spaCy
* Extract Nouns, Proper nouns, compound nouns (aspects)
* Extract adjectives, CCONJ, SCONJ (opinion words)
* Extract polarity label based on avg. of opinion words + aspects (multilingual senticnet)

In [ ]:
## INPUT: text
## OUTPUT: dictionary with modifiers, chunks, noun_polarity (Sentic), modifier polarity

def output_dictionary(txt):
  #initialize dictionary
  noun_adj_pairs_en = {}
  #create spacy doc element
  doc = nlp(txt)

  for chunk in doc.noun_chunks:
    adj = []
    mod_polarity = []
    comps = []
    noun = ""

    for tok in chunk:

      # run check to see if word = compound. Otherwise the code will limit the words to just one.
      if tok.pos_ in ["NOUN", "PROPN"] and tok.lemma_ not in nlp.Defaults.stop_words and len(tok.lemma_) > 3:  #nouns and proper nouns

        if tok.dep_ == "compound":

          noun = doc[tok.i: tok.head.i + 1].lemma_ #take the compound noun
          noun = noun.rstrip() #remove newline
          comps.append(tok.head.i) #add piece of the compound noun which you added to the index list
          noun_polarity = return_polarity_scores(noun)

        else:
          if tok.i not in comps: #if the word is not already added to the index list (because it is part of a compound noun)
            noun = tok.lemma_
            noun = noun.rstrip()
            noun_polarity = return_polarity_scores(noun)


      if tok.pos_ == "ADJ" or tok.pos_ == "CCONJ" or tok.pos_ == "SCONJ": #adjectives, coordinating conjunction, subordinating conjunction
        if tok.text != "and": #don't add "and" (coordinating conjunction word)
          adj.append(tok.text)
          modifier_polarity = return_polarity_scores(tok.text)
          mod_polarity.append(modifier_polarity) #add modifier polarity score to list

    if noun:
        noun_adj_pairs_en.update({noun: {"modifiers": adj, "chunk": chunk, "noun_polarity": noun_polarity, "modifier_polarity": mod_polarity}}) #add all modifiers and chunks, noun polarity and modifier polarity


  return noun_adj_pairs_en

In [ ]:
def return_polarity_scores(word):
  try:
    polarity_value = bsn.polarity_value(word) #try to find the word in the multilingual senticnet
  except KeyError:
    try:
      polarity_value = sn.polarity_value(word) #try to find the word in the English senticnet
    except KeyError:
      polarity_value = 0 #if not found, return 0 (neutral)
  return float(polarity_value)

In [ ]:
#normalize scores with sigmoid in 0-1 range
def sig(x):
 return 1/(1 + np.exp(-x))

In [ ]:
def polarity_label(score): #add polarity label based on Senticnet score

  if score <= 0.20:
    return 1
  elif score > 0.20 and score <= 0.40:
    return 2
  elif score > 0.40 and score <= 0.60:
    return 3
  elif score > 0.60 and score <= 0.80:
    return 4
  elif score > 0.80 and score <= 1:
    return 5

In [ ]:
def add_mean_polarity_score(noun_adj_pairs):
  for k, v in noun_adj_pairs.items():
    if v['modifier_polarity']:
      #take the mean polarity scores of the modifiers + push in a 0-1 range w/ sigmoid
      #because the range of Sentic is -1 : 1
      mean_pol = sig(statistics.mean([float(x) for x in v["modifier_polarity"]]))
      label = polarity_label(mean_pol)

      noun_adj_pairs_en[k]["mean_polarity"] = mean_pol
      noun_adj_pairs_en[k]["polarity_label"] = label #add a polarity label according to the gold standard annotations

  return noun_adj_pairs

In [ ]:
noun_adj_pairs_en = output_dictionary(test_text)
result = add_mean_polarity_score(noun_adj_pairs_en)

In [ ]:
result

# ASPECTS: prep gold standard dataframe
*  make dataframe into bio-labels (aspects)

In [ ]:
f = "/content/drive/MyDrive/1. WORK IN PROGRESS/PhD/ABSA notebooks/travel_literature17674947568396187814.zip"

In [ ]:
project = Project.from_zipped_xmi(f)

In [ ]:
annotators = project.annotators #Exclude Fred from annotations

annotators.pop(0)
annotators.pop(-1)

'Fred_Bondue'

In [ ]:
FILES  = [f for f in project.source_file_names if "IAA" in f] # take IAA test files (annotated by all)
cats = ["ASP_SENT_NULL", "entitycategory", "sentimentcategory"]

In [ ]:
pos_type= "webanno.custom.Emotions_Entities"

In [ ]:
feature_path = f'{pos_type}>{"entitycategory"}'
pos_annos = project.select(annotation=feature_path) #project = defined outside
annos_sents = pos_annos.data_frame
annos = annos_sents[annos_sents["annotation"] != "None"]

In [ ]:
#annos contains the annotations across all the languages.
#We only want the English ones, so we'll filter (names contain English or IT)

annos_en = annos[(annos['source_file'].str.contains("English")) | (annos['source_file'].str.contains("IT"))]

In [ ]:
annos_en["annotation"].unique()

array(['PERSON', 'LOCATION', 'WEATHER', 'NAT_PHENOMENON', 'FAUNA',
       'NAT_LANDFORM', 'LAND_COVER', 'HUM_LANDFORM', 'FLORA',
       'ORGANISATION', 'MYTH', 'BIOME', 'NULL'], dtype=object)

In [ ]:
annos_en['source_file'].unique()

array(['GB-117_sample_English_18.txt', 'IT_5_sample.txt',
       'GB-91_sample_English_20.txt', 'GB-138_sample_English_20.txt',
       'BHL_118_sample_English_20.0.txt',
       'BHL_1949_sample_English_18.0.txt', 'GB-70_sample_English_19.txt',
       'GB-104_sample_English_20.txt', 'GB-7_sample_English_18.txt',
       'GB-116_sample_English_18.txt', 'GB-111_sample_IAA_20_English.txt',
       'IT_30_sample.txt', 'IT_2_sample.txt', 'IT_19_test.txt',
       'BHL_1903_sample_English_18.0.txt', 'GB-4_sample_English_18.txt',
       'GB-111_sample_English_20.txt', 'GB-58_sample_English_19.txt',
       'GB-29_sample_English_19.txt', 'GB-22_sample_IAA_19_English.txt',
       'GB-108_sample_English_20.txt', 'GB-78_sample_English_19.txt'],
      dtype=object)

In [ ]:
annos_en["annotation"].value_counts()

PERSON            2103
LOCATION          1672
HUM_LANDFORM      1393
NAT_LANDFORM      1077
FAUNA              488
FLORA              390
WEATHER            305
ORGANISATION       219
NAT_PHENOMENON     208
LAND_COVER         200
BIOME               71
MYTH                43
NULL                 1
Name: annotation, dtype: int64

In [ ]:
len(annos_en)

8170

In [ ]:
annos_en.head()

,source_file,sentence,begin,end,annotator,_annotation,text,_sentence_text,annotation
0,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_0-111,8,18,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",James Cook,Author: James Cook\nTitle: James Cook\nLanguag...,PERSON
1,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_0-111,68,79,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",NEW ZEALAND,Author: James Cook\nTitle: James Cook\nLanguag...,LOCATION
2,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_0-111,83,94,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",NEW HOLLAND,Author: James Cook\nTitle: James Cook\nLanguag...,LOCATION
3,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_112-166,117,128,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",New Zealand,"From New Zealand to Australia . ] SUNDAY , 1st...",LOCATION
4,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_112-166,132,141,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",Australia,"From New Zealand to Australia . ] SUNDAY , 1st...",LOCATION


In [ ]:
annos_en.to_csv("/content/drive/MyDrive/1. WORK IN PROGRESS/PhD/ABSA notebooks/data/categories_inception_raw.csv")

In [ ]:
df1 = (annos_en.set_index(['text'], append=True)
         ._sentence_text.str.split(expand=True) #split sentences into words based on spaces and expand dataframe based on every single token
         .stack()
         .reset_index(level=2, drop=True)
         .reset_index(name="Word")
         .rename(columns={'level_0': 'Sentence'})) #stack on top of one another

In [ ]:
df1['Sentence'] = 'Sentence: ' + df1['Sentence'].astype(str)  #make an index string object
w = df1['Word'].str.replace(r'[^\w\s]+', '') #dataframe w/ conditional: if the word begins with a whitespace or any other character, word = gone
splitted = df1.pop('text').str.split() #split aspect text on space
m1 = splitted.str[0].eq(w) # filter : check if the first element of the "splitted" dataframe is equal to the word token
m2 = [b in a for a, b in zip(splitted, w)] #check if the word is in the "splitted" element (["stark", "geregnet"] --> "geregnet" --> TRUE)

df1["entity"] = "aspect"
e = df1.pop('entity')

<ipython-input-21-a9630f570f5c>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  w = df1['Word'].str.replace(r'[^\w\s]+', '') #dataframe w/ conditional: if the word begins with a whitespace or any other character, word = gone


In [ ]:
df1['Entity'] = np.select([m1, m2 & ~m1], ['B-' + e, 'I-' + e],  default='O')

#m1 = condition --> "B" + e
#m2 and not m1 (not the beginning of the word, but the word is in the splitted collection)= condition --> "I" + e

In [ ]:
df1 #English annotations

,Sentence,Word,Entity
0,Sentence: 0,Author:,O
1,Sentence: 0,James,B-aspect
2,Sentence: 0,Cook,I-aspect
3,Sentence: 0,Title:,O
4,Sentence: 0,James,B-aspect
...,...,...,...
382021,Sentence: 19418,at,O
382022,Sentence: 19418,Fort,B-aspect
382023,Sentence: 19418,Niagara,I-aspect
382024,Sentence: 19418,",",O


In [ ]:
group = df1.groupby(["Sentence"], axis = 0, as_index= False)

In [ ]:
# function to concatenate words and entities into lists of strings
#compare "word" with one another --> if same then perform operation on entity change O to aspects
# expand again

In [ ]:
df1.keys()[0]

'Sentence'

In [ ]:
#t= df1[(df1['Sentence'] == "Sentence: 0") | (df1['Sentence'] == "Sentence: 1")]

In [ ]:
labelled_sentences = {}


for index, group in df1.groupby("Sentence"):
  indexes_past = [] #keep track of the index ("Sentence: 0", ...)

  if index not in indexes_past:
    words = [w for w in group["Word"].tolist()] #grab all the words
    labels = [l for l in group["Entity"].tolist()]

    labelled_sentences[index] = {"words": words, "labels": labels}

    indexes_past.append(index)

In [ ]:
lab_df = pd.DataFrame.from_dict(labelled_sentences, orient='index')

In [ ]:
lab_df.keys()

Index(['words', 'labels'], dtype='object')

In [ ]:
def join_lists(l_of_words):
  return " ".join(l_of_words)

In [ ]:
lab_df['sentence'] = lab_df["words"].apply(join_lists)

In [ ]:
lab_df['sent_id'] = lab_df.groupby('sentence').ngroup() #add a sentence id to rows with the same sentence

In [ ]:
x = lab_df.groupby("sent_id")

In [ ]:
type(x)

pandas.core.groupby.generic.DataFrameGroupBy

In [ ]:
#debug = lab_df.sort_values(by = ["sent_id"])[(lab_df['sent_id'] == 2176) | (lab_df['sent_id'] == 5)].groupby('sent_id') #run check

<ipython-input-70-68049b8456a5>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  debug = lab_df.sort_values(by = ["sent_id"])[(lab_df['sent_id'] == 2176) | (lab_df['sent_id'] == 5)].groupby('sent_id')


In [ ]:
all_new_labels = []
for i in x.labels:
  t = i[-1].tolist() #labels
  id = i[0] #id

  new_id_labels = {}

  # if there is only 1 annotation for the sentence (t), we don't want to perform zip bc ["O", "B-aspect"] --> ("O", "B-aspect")
  if len(t) == 1:
    new_id_labels["sent_id"] = id
    new_id_labels["labels"] = t[0]

  else: #if there's multiple annotations to assemble
    # Assemble new label list based on the merged labels for all the sentences
    new_labels = []
    for el in zip(*t): #make tuples from every list element at the same index
      if len(set(el)) > 1:
        label = [lab for lab in set(el) if lab != 'O'][0] #take label which is not 'O'
        new_labels.append(label)
      else:
        label = "O" #if label is not B-aspect or I-aspect, it's automatically O
        new_labels.append(label)

    new_id_labels["sent_id"] = id
    new_id_labels["labels"] = new_labels
  all_new_labels.append(new_id_labels)

print(all_new_labels)

[{'sent_id': 0, 'labels': ['O', 'O', 'O', 'O', 'B-aspect', 'B-aspect', 'O', 'O', 'O', 'B-aspect', 'O', 'O', 'O', 'B-aspect', 'O', 'O', 'B-aspect', 'I-aspect', 'O', 'O', 'B-aspect', 'O', 'O', 'B-aspect', 'O']}, {'sent_id': 1, 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'B-aspect', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}, {'sent_id': 2, 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'B-aspect', 'I-aspect', 'I-aspect', 'O']}, {'sent_id': 3, 'labels': ['O', 'O', 'O', 'B-aspect']}, {'sent_id': 4, 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-aspect', 'O', 'O', 'O']}, {'sent_id': 5, 'labels': ['O', 'O', 'B-aspect', 'I-aspect', 'I-aspect', 'O', 'B-aspect', 'B-aspect', 'I-aspect', 'B-aspect', 'I-aspect', 'I-aspect', 'B-aspect', 'I-aspect', 'I-aspect', 'B-aspect', 'I-aspect', 'B-aspect', 'O', 'I-aspect', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-aspect', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-aspect',

In [ ]:
lab_df_new = pd.DataFrame(all_new_labels)

In [ ]:
lab_df.keys()

Index(['words', 'labels', 'sentence', 'sent_id'], dtype='object')

In [ ]:
# merge lab_df and new labels based on id
# remove duplicates
#overwrite lab_df

gold = lab_df.merge(lab_df_new, on= "sent_id").sort_values(by="sent_id").drop_duplicates(subset = ["sent_id"]).drop(["labels_x"], axis= 1).reset_index(drop = True)

In [ ]:
gold

,words,sentence,sent_id,labels_y
0,"['', CHAPTER, III, THE, CUMÆAN, SIBYL, A, part...",'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,"[O, O, O, O, B-aspect, B-aspect, O, O, O, B-as..."
1,"['', This, ,, and, ``, May, God, forgive, the,...","'' This , and `` May God forgive the sins of y...",1,"[O, O, O, O, O, O, B-aspect, O, O, O, O, O, O,..."
2,"['', are, surely, the, utterances, of, a, tire...",'' are surely the utterances of a tired race .,2,"[O, O, O, O, O, O, B-aspect, I-aspect, I-aspec..."
3,"[A, Belgian, ,, M.]","A Belgian , M.",3,"[O, O, O, B-aspect]"
4,"[A, ``, third, edition, ,, revised, and, enlar...","A `` third edition , revised and enlarged , ''...",4,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...,...
2174,"[my, journey, from, masuah, to, gondar, transa...",my journey from masuah to gondar transactions ...,2174,"[O, O, O, B-aspect, O, B-aspect, O, O, O, O, O..."
2175,"[preserved, ], the, spirit, of, the, Indians, ...",preserved ] the spirit of the Indians and kept...,2175,"[O, O, O, O, O, O, B-aspect, O, O, O, O, O, O,..."
2176,"[the, Zibib, !]",the Zibib !,2176,"[O, B-aspect, O]"
2177,"[then, ,, in, his, own, language, of, Tigre, ,...","then , in his own language of Tigre , he afked...",2177,"[O, O, O, O, O, O, O, B-aspect, O, O, O, O, O,..."


In [ ]:
gold_final = gold.explode(["words", "labels_y"])

In [ ]:
gold_final

,words,sentence,sent_id,labels_y
0,'','' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
0,CHAPTER,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
0,III,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
0,THE,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
0,CUMÆAN,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,B-aspect
...,...,...,...,...
2178,up,— I am inclined to believe that a confcioufnef...,2178,O
2178,arms,— I am inclined to believe that a confcioufnef...,2178,O
2178,againft,— I am inclined to believe that a confcioufnef...,2178,O
2178,him,— I am inclined to believe that a confcioufnef...,2178,O


In [ ]:
#gold_final.to_csv("/content/drive/MyDrive/1. WORK IN PROGRESS/PhD/ABSA notebooks/gold_standard_aspects.csv")

# ASPECTS: evaluate aspect extraction method

While (using spaCy):


*   Extract noun chunks which contain a Noun and a Proper Noun
*   NOTE: aspect data is in categories as defined in the annotation guidelines. The approach here extracts all the chunks without categorization - so we look at recall to evaluate it (how many of the annotated aspects were found by the approach)



In [ ]:
gold = pd.read_csv("/content/drive/MyDrive/1. WORK IN PROGRESS/PhD/ABSA notebooks/gold_standard_aspects.csv")

In [ ]:
gold.head()

,Unnamed: 0,words,sentence,sent_id,labels_y
0,0,'','' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
1,0,CHAPTER,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
2,0,III,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
3,0,THE,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
4,0,CUMÆAN,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,B-aspect


In [ ]:
import re
from spacy.tokenizer import Tokenizer

In [ ]:
test_sentence = gold.iloc[0]["sentence"]

In [ ]:
nlp.tokenizer = Tokenizer(nlp.vocab, token_match=re.compile(r'\S+').match) #add rule to tokenizer to tokenize text based on whitespace to match the gold standard tokens

In [ ]:
def get_aspect_labels(txt):

  doc = nlp(txt)
  chunks = [chunk for chunk in doc.noun_chunks for tok in chunk if tok.pos_ in ["NOUN", "PROPN"] and tok.lemma_ not in nlp.Defaults.stop_words] #return chunk if it contains a noun/proper noun and is not made up of stop words

  tokens = ["O" for tok in doc]

  for chunk in chunks:
    len_chunk = len([tok for tok in chunk])

    if len_chunk > 1: #if the chunk has more than 1 token
      indices = [tok.i for tok in chunk] #indices of the chunk tokens
      tokens[indices[0]] = "B-aspect" #the first element of the chunk = B-aspect

      for ind in indices[1::]:
        tokens[ind] = "I-aspect" #the other elements of the chunk = I-aspect

    else: #if the chunk just has one element, = B-aspect
      indices = [tok.i for tok in chunk]
      tokens[indices[0]] = "B-aspect"

  return tokens


In [ ]:
def get_tokens(txt):
  return [tok for tok in doc]

In [ ]:
def get_chunks(txt):
  doc = nlp(txt)
  chunks = [chunk for chunk in doc.noun_chunks for tok in chunk if tok.pos_ in ["NOUN", "PROPN"] and tok.lemma_ not in nlp.Defaults.stop_words] #return chunk if it contains a noun/proper noun and is not made up of stop words

  return chunks

In [ ]:
gold["chunks"] = gold["sentence"].apply(get_chunks)

In [ ]:
gold.to_csv("/content/drive/MyDrive/1. WORK IN PROGRESS/PhD/ABSA notebooks/data/rule_based_annotations.csv")

In [ ]:
gold

,Unnamed: 0,words,sentence,sent_id,labels_y,predicted_label,chunks
0,0,'','' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O,"[O, B-aspect, I-aspect, I-aspect, I-aspect, O,...","[(CHAPTER, III, THE, CUMÆAN), (CHAPTER, III, T..."
1,0,CHAPTER,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O,"[O, B-aspect, I-aspect, I-aspect, I-aspect, O,...","[(CHAPTER, III, THE, CUMÆAN), (CHAPTER, III, T..."
2,0,III,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O,"[O, B-aspect, I-aspect, I-aspect, I-aspect, O,...","[(CHAPTER, III, THE, CUMÆAN), (CHAPTER, III, T..."
3,0,THE,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O,"[O, B-aspect, I-aspect, I-aspect, I-aspect, O,...","[(CHAPTER, III, THE, CUMÆAN), (CHAPTER, III, T..."
4,0,CUMÆAN,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,B-aspect,"[O, B-aspect, I-aspect, I-aspect, I-aspect, O,...","[(CHAPTER, III, THE, CUMÆAN), (CHAPTER, III, T..."
...,...,...,...,...,...,...,...
76747,2178,up,— I am inclined to believe that a confcioufnef...,2178,O,"[O, O, O, O, O, O, O, B-aspect, I-aspect, O, B...","[(a, confcioufnefs), (his, ovv^n, ftrength), (..."
76748,2178,arms,— I am inclined to believe that a confcioufnef...,2178,O,"[O, O, O, O, O, O, O, B-aspect, I-aspect, O, B...","[(a, confcioufnefs), (his, ovv^n, ftrength), (..."
76749,2178,againft,— I am inclined to believe that a confcioufnef...,2178,O,"[O, O, O, O, O, O, O, B-aspect, I-aspect, O, B...","[(a, confcioufnefs), (his, ovv^n, ftrength), (..."
76750,2178,him,— I am inclined to believe that a confcioufnef...,2178,O,"[O, O, O, O, O, O, O, B-aspect, I-aspect, O, B...","[(a, confcioufnefs), (his, ovv^n, ftrength), (..."


In [ ]:
gold["predicted_label"] = gold["sentence"].apply(get_aspect_labels)

In [ ]:
gold

,Unnamed: 0,words,sentence,sent_id,labels_y,predicted_label
0,0,'','' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O,"[O, B-aspect, I-aspect, I-aspect, I-aspect, O,..."
1,0,CHAPTER,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O,"[O, B-aspect, I-aspect, I-aspect, I-aspect, O,..."
2,0,III,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O,"[O, B-aspect, I-aspect, I-aspect, I-aspect, O,..."
3,0,THE,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O,"[O, B-aspect, I-aspect, I-aspect, I-aspect, O,..."
4,0,CUMÆAN,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,B-aspect,"[O, B-aspect, I-aspect, I-aspect, I-aspect, O,..."
...,...,...,...,...,...,...
76747,2178,up,— I am inclined to believe that a confcioufnef...,2178,O,"[O, O, O, O, O, O, O, B-aspect, I-aspect, O, B..."
76748,2178,arms,— I am inclined to believe that a confcioufnef...,2178,O,"[O, O, O, O, O, O, O, B-aspect, I-aspect, O, B..."
76749,2178,againft,— I am inclined to believe that a confcioufnef...,2178,O,"[O, O, O, O, O, O, O, B-aspect, I-aspect, O, B..."
76750,2178,him,— I am inclined to believe that a confcioufnef...,2178,O,"[O, O, O, O, O, O, O, B-aspect, I-aspect, O, B..."


In [ ]:
eval = gold.explode(["labels_y", "predicted_label", "words"])

In [ ]:
gold.keys()

In [ ]:
true = gold["labels_y"].to_list()
predicted = gold["predicted_label"].to_list()

In [ ]:
evaluator = Evaluator(true, predicted, tags=['aspect'], loader="list")

results, results_by_tag = evaluator.evaluate()

In [ ]:
results

# OPINION: create gold standard dataframe (opinion words)




In [ ]:
feature_path = f'{pos_type}>{"sentimentcategory"}'
pos_annos = project.select(annotation=feature_path) #project = defined outside
annos_sents = pos_annos.data_frame
annos = annos_sents[annos_sents["annotation"] != "None"]

In [ ]:
annos_en = annos[(annos['source_file'].str.contains("English")) | (annos['source_file'].str.contains("IT"))] #only English annots

In [ ]:
def to_str(txt):
  return str(txt)

In [ ]:
#Get rid of NULL (cast annotation to str)
annos_en["t"] = annos_en["_annotation"].apply(to_str)

<ipython-input-89-753b760bff0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annos_en["t"] = annos_en["_annotation"].apply(to_str)


In [ ]:
annos_en = annos_en[~annos_en["t"].str.contains("NULL=NULL")].drop(["t", "annotator"], axis = 1)

In [ ]:
annos_en.head(50)

,source_file,sentence,begin,end,_annotation,text,_sentence_text,annotation
5,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_167-303,185,193,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",moderate,"In the P.M. had a moderate breeze at East , wh...",3
16,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_1171-1330,1254,1258,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",fair,At Midnight the wind came to South-South-West ...,4
21,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_1480-1687,1542,1547,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",fresh,Cloudy weather ; Winds at South-West and South...,4
23,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_1480-1687,1555,1589,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",with which we made our Course good,Cloudy weather ; Winds at South-West and South...,4
24,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_1826-1944,1832,1844,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",steady brisk,Had a steady brisk Gale at South-South-West wi...,4
28,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_2283-2356,2283,2288,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",Fresh,"Fresh Gales at South , which in the A.M. veer ...",4
33,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_2945-2994,2945,2951,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",Gentle,"Gentle breezes at North-East , which in the A.M .",4
36,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_3250-3375,3287,3310,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",refusing to do his duty,"Bowles , Marine , with 12 lashes for refusing ...",2
38,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_3959-4025,3959,3965,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",Gentle,Gentle breezes at North-West ; pleasant weathe...,4
40,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_3959-4025,3990,3998,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",pleasant,Gentle breezes at North-West ; pleasant weathe...,4


In [ ]:
df1 = (annos_en.set_index(['text'], append=True)
         ._sentence_text.str.split(expand=True) #split sentences into words based on spaces and expand dataframe based on every single token
         .stack()
         .reset_index(level=2, drop=True)
         .reset_index(name="Word")
         .rename(columns={'level_0': 'Sentence'})) #stack on top of one another

In [ ]:
df1.keys()

Index(['Sentence', 'text', 'Word'], dtype='object')

In [ ]:
df1['Sentence'] = 'Sentence: ' + df1['Sentence'].astype(str)  #make an index string object

In [ ]:
df1

,Sentence,text,Word
0,Sentence: 5,moderate,In
1,Sentence: 5,moderate,the
2,Sentence: 5,moderate,P.M.
3,Sentence: 5,moderate,had
4,Sentence: 5,moderate,a
...,...,...,...
63614,Sentence: 19396,destroyed it,the
63615,Sentence: 19396,destroyed it,case
63616,Sentence: 19396,destroyed it,of
63617,Sentence: 19396,destroyed it,Capt


In [ ]:
len(df1)

63619

In [ ]:
w = df1['Word']

In [ ]:
w

0          In
1         the
2        P.M.
3         had
4           a
         ... 
63614     the
63615    case
63616      of
63617    Capt
63618       .
Name: Word, Length: 63619, dtype: object

In [ ]:
splitted = df1.pop('text').str.split()

In [ ]:
splitted

0             [moderate]
1             [moderate]
2             [moderate]
3             [moderate]
4             [moderate]
              ...       
63614    [destroyed, it]
63615    [destroyed, it]
63616    [destroyed, it]
63617    [destroyed, it]
63618    [destroyed, it]
Name: text, Length: 63619, dtype: object

In [ ]:
m1 = splitted.str[0].eq(w) # filter : check if the first element of the "splitted" list of opinion words is equal to the word token

In [ ]:
m1

0        False
1        False
2        False
3        False
4        False
         ...  
63614    False
63615    False
63616    False
63617    False
63618    False
Length: 63619, dtype: bool

In [ ]:
m2 = [b in a for a, b in zip(splitted, w)] #check if the word is in the "splitted" element (["stark", "geregnet"] --> "geregnet" --> TRUE)

In [ ]:
df1["opinion"] = "opinion"

In [ ]:
e = df1.pop('opinion')

In [ ]:
df1['Opinion'] = np.select([m1, m2 & ~m1], ['B-' + e, 'I-' + e],  default='O') #m1 = first element of the list is B + opinion, other elements of the list are I + opinion if not equal to first element

In [ ]:
#df1.drop("_sentence_text_list_all", inplace=True, axis = 1)

In [ ]:
group = df1.groupby(["Sentence"], axis = 0, as_index= False) #group the dataframe by sentence

In [ ]:
labelled_sentences = {}


for index, group in df1.groupby("Sentence"):
  indexes_past = [] #keep track of the index ("Sentence: 0", ...)

  if index not in indexes_past:
    words = [w for w in group["Word"].tolist()] #grab all the words
    labels = [l for l in group["Opinion"].tolist()]

    labelled_sentences[index] = {"words": words, "labels": labels}

    indexes_past.append(index)

In [ ]:
lab_df = pd.DataFrame.from_dict(labelled_sentences, orient='index')
lab_df['sentence'] = lab_df["words"].apply(join_lists)
lab_df['sent_id'] = lab_df.groupby('sentence').ngroup() #add a sentence id to rows with the same sentence
x = lab_df.groupby("sent_id")

In [ ]:
all_new_labels = []
for i in x.labels:
  t = i[-1].tolist() #labels
  id = i[0] #id

  new_id_labels = {}

  # if there is only 1 annotation for the sentence (t), we don't want to perform zip bc ["O", "B-aspect"] --> ("O", "B-aspect")
  if len(t) == 1:
    new_id_labels["sent_id"] = id
    new_id_labels["labels"] = t[0]

  else: #if there's multiple annotations to assemble
    # Assemble new label list based on the merged labels for all the sentences
    new_labels = []
    for el in zip(*t): #make tuples from every list element at the same index
      if len(set(el)) > 1:
        label = [lab for lab in set(el) if lab != 'O'][0] #take label which is not 'O'
        new_labels.append(label)
      else:
        label = "O" #if label is not B-aspect or I-aspect, it's automatically O
        new_labels.append(label)

    new_id_labels["sent_id"] = id
    new_id_labels["labels"] = new_labels
  all_new_labels.append(new_id_labels)

print(all_new_labels)

[{'sent_id': 0, 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-opinion', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}, {'sent_id': 1, 'labels': ['O', 'B-opinion', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-opinion', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}, {'sent_id': 2, 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-opinion', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-opinion', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [ ]:
lab_df_new = pd.DataFrame(all_new_labels)

In [ ]:
gold = lab_df.merge(lab_df_new, on= "sent_id").sort_values(by="sent_id").drop_duplicates(subset = ["sent_id"]).drop(["labels_x"], axis= 1).reset_index(drop = True)

In [ ]:
gold.explode(["words", "labels_y"])

,words,sentence,sent_id,labels_y
0,'','' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
0,CHAPTER,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
0,III,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
0,THE,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
0,CUMÆAN,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
...,...,...,...,...
803,up,— I am inclined to believe that a confcioufnef...,803,O
803,arms,— I am inclined to believe that a confcioufnef...,803,O
803,againft,— I am inclined to believe that a confcioufnef...,803,O
803,him,— I am inclined to believe that a confcioufnef...,803,O


In [ ]:
def fix_labels(label_list):
  for ind, el in enumerate(label_list):
    if el == "I-opinion" and label_list[ind-1] == "O":
      label_list[ind] = "O"
    #except IndexError:
      #print(label_list)

  return label_list

In [ ]:
gold["new_labels"] = gold["labels_y"].apply(fix_labels)

In [ ]:
gold.head()

,words,sentence,sent_id,labels_y,new_labels
0,"['', CHAPTER, III, THE, CUMÆAN, SIBYL, A, part...",'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,"[O, O, O, O, O, O, O, O, O, O, B-opinion, O, O...","[O, O, O, O, O, O, O, O, O, O, B-opinion, O, O..."
1,"[A, big, heap, of, stones, marked, the, summit...",A big heap of stones marked the summit of the ...,1,"[O, B-opinion, O, O, O, O, O, O, O, O, O, O, O...","[O, B-opinion, O, O, O, O, O, O, O, O, O, O, O..."
2,"[A, blazing, hillside, ;, a, mountain, of, gra...",A blazing hillside ; a mountain of gravel amon...,2,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[A, brief, visit, to, Seymour, Island, did, no...",A brief visit to Seymour Island did not reveal...,3,"[O, O, O, O, O, O, B-opinion, I-opinion, I-opi...","[O, O, O, O, O, O, B-opinion, I-opinion, I-opi..."
4,"[A, color, suffusion, is, common, throughout, ...",A color suffusion is common throughout the ent...,4,"[O, O, O, O, B-opinion, O, O, O, O, O, O]","[O, O, O, O, B-opinion, O, O, O, O, O, O]"


# OPINION: evaluate opinion word extraction method

While:

*   Extract opinion words (adjectives, coordinating conjunction, subordinating
 conjunction) from noun phrase
*   Extract adjectives NOUN - AUX - ADJ structures ("the house is beautiful")
*   Extract amplifiers (very, extremely, ...) --> influence score
*   Take negations into account (if "not" before adj --> get antonym through wordnet synsets)
*   Evaluate on gold standard data



In [ ]:
matcher = Matcher(nlp.vocab)

In [ ]:
pattern = [ [[{"POS": "AUX"}, {"DEP": "neg", "OP": "*"}, {"POS": "ADV", "OP": "*"}, {"POS": "ADJ"}]] ] #is + (not) + (very) + nice
pattern_neg_conj =  [ [[{"POS": "AUX", "OP": "{0}"}, {"POS": "ADV", "OP": "{0}"}, {"POS": "ADJ", "OP": "{0}"}, {"POS": "CCONJ"},  {"DEP": "neg"}, {"POS": "ADJ"}]]  ] # [is + (very) + nice]DONOTMATCH + but + not + warm

matcher.add("aux_adv_adj", pattern[0])
matcher.add("negations_cconj", pattern_neg_conj[0])

In [ ]:
def match_auxiliary_phrases(doc):
  spans = []

  matches = matcher(doc)

  for match_id, start, end in matches:
    span = [x for x in range(start,end)]
    spans.append(span[1::]) #auxiliary verb 'is' or "cconj" "but"/... = not important

  return spans

In [ ]:
# Fetch adjectives in noun chunks

def opinion_extractor(txt):
  doc = nlp(txt)
  tokens = ["O" for tok in doc] #initialize token list, length of list = all "O"s


  ### AUXILIARY PHRASES ###

  #fetch auxiliary constructions (the house *is very nice*)
  auxiliary_spans = match_auxiliary_phrases(doc) #get indices of spans auxiliary sentences
  for span in auxiliary_spans:
    tokens[span[0]] = "B-opinion"
    for span_ind in span[1::]:
      tokens[span_ind] = "I-opinion"


  ### NOUN CHUNKS ###

  #return chunk if it contains a noun/proper noun and is not made up of stop words
  all_modifier_indices = []
  for chunk in doc.noun_chunks:
    #print(chunk)

    for tok in chunk:
      if tok.dep_ == "compound":
        continue
      elif tok.pos_ in ["ADJ", "CCONJ", "SCONJ"] and tok.lemma_ not in nlp.Defaults.stop_words:
        modifier_indices = []
        modifier_index = tok.i
        modifier_indices.append(modifier_index) #save index of adjectives looped over



  # Fetch intensifiers by navigating children (adapted from https://towardsdatascience.com/aspect-based-sentiment-analysis-using-spacy-textblob-4c8de3e0d2b9)
        for child in tok.children:
          if child.pos_ == "ADV":
            intensifier_index = child.i
            modifier_indices.append(intensifier_index)
          elif child.dep_ == "neg": #fetch negations to account for negations
            intensifier_index = child.i
            modifier_indices.append(intensifier_index)

        all_modifier_indices.append(sorted(modifier_indices))


  for mod_pair in all_modifier_indices:
    if len(mod_pair) > 1:
      tokens[mod_pair[0]] = "B-opinion"
      for opinion_id in mod_pair[1::]:
        tokens[opinion_id] = "I-opinion"

    else:
      tokens[mod_pair[0]] = "B-opinion"

  return tokens

In [ ]:
gold["predicted_label"] = gold["sentence"].apply(opinion_extractor)

In [ ]:
gold.explode(["labels_y", "predicted_label", "words"])

,words,sentence,sent_id,labels_y,new_labels,predicted_label
0,'','' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O,"[O, O, O, O, O, O, O, O, O, O, B-opinion, O, O...",O
0,CHAPTER,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O,"[O, O, O, O, O, O, O, O, O, O, B-opinion, O, O...",O
0,III,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O,"[O, O, O, O, O, O, O, O, O, O, B-opinion, O, O...",O
0,THE,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O,"[O, O, O, O, O, O, O, O, O, O, B-opinion, O, O...",O
0,CUMÆAN,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O,"[O, O, O, O, O, O, O, O, O, O, B-opinion, O, O...",O
...,...,...,...,...,...,...
803,up,— I am inclined to believe that a confcioufnef...,803,O,"[O, O, O, O, O, O, O, O, O, O, B-opinion, O, O...",O
803,arms,— I am inclined to believe that a confcioufnef...,803,O,"[O, O, O, O, O, O, O, O, O, O, B-opinion, O, O...",O
803,againft,— I am inclined to believe that a confcioufnef...,803,O,"[O, O, O, O, O, O, O, O, O, O, B-opinion, O, O...",O
803,him,— I am inclined to believe that a confcioufnef...,803,O,"[O, O, O, O, O, O, O, O, O, O, B-opinion, O, O...",O


In [ ]:
eval = gold.explode(["labels_y", "predicted_label", "words"])
true = gold["labels_y"].to_list()
predicted = gold["predicted_label"].to_list()

In [ ]:
evaluator = Evaluator(true, predicted, tags=['opinion'], loader="list")

results, results_by_tag = evaluator.evaluate()

In [ ]:
results

{'ent_type': {'correct': 767,
  'incorrect': 0,
  'partial': 0,
  'missed': 617,
  'spurious': 1207,
  'possible': 1384,
  'actual': 1974,
  'precision': 0.38855116514690985,
  'recall': 0.5541907514450867,
  'f1': 0.45681953543776055},
 'partial': {'correct': 406,
  'incorrect': 0,
  'partial': 361,
  'missed': 617,
  'spurious': 1207,
  'possible': 1384,
  'actual': 1974,
  'precision': 0.29711246200607905,
  'recall': 0.42377167630057805,
  'f1': 0.34931506849315075},
 'strict': {'correct': 406,
  'incorrect': 361,
  'partial': 0,
  'missed': 617,
  'spurious': 1207,
  'possible': 1384,
  'actual': 1974,
  'precision': 0.20567375886524822,
  'recall': 0.29335260115606937,
  'f1': 0.24181060154854078},
 'exact': {'correct': 406,
  'incorrect': 361,
  'partial': 0,
  'missed': 617,
  'spurious': 1207,
  'possible': 1384,
  'actual': 1974,
  'precision': 0.20567375886524822,
  'recall': 0.29335260115606937,
  'f1': 0.24181060154854078}}

# SENTIMENT LABEL: evaluate sentiment score allotted to opinion word

* Apply senticnet approach to opinion words of gold standard data
* + apply to automatically sourced opinion words
* Make sure that the intensifiers influence the score
* Take negations into account
  - swap the scores of the adjective (5 = 1, 4 = 2)

In [ ]:
#when we find a negation, we want to apply the scoring to the antonym of the word in question
def fetch_antonym(word):
  antonyms = []

  try:
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            if l.antonyms():
                antonyms.append(l.antonyms()[0].name())
    return antonyms[0]
  except:
    return False
   #return first element in antonyms list

In [ ]:
inverses = {"1":5, "2":4, "3":3, "4": 2, "5": 1}

In [ ]:
# Apply sentic scorer to all of the words in the gold standard data
# if negation: turn word into antonym using wordnet OR swap the scores
# return mean

def sentiment_scorer(text):
  doc = nlp(text)

  text = [tok.lemma_ for tok in doc if not tok.is_punct]

  print(text)

  ## Convert negations to antonyms ###

  negation_tokens = [tok for tok in doc if tok.dep_ == 'neg']
  negation_head_tokens = [(token.head, token.head.i) for token in negation_tokens] #get dependency head (= negated word)


  inverse_scores = []
  for w in negation_head_tokens:
    word = str(w[0])
    ind = w[1] #index of token

    antonym = fetch_antonym(word)

    if not antonym: #if an antonym cannot be found, find the opposite label

      word_label = polarity_label(sig(return_polarity_scores(word)))
      print(word_label)
      inverse_score = [*map(inverses.get, str(word_label))][0]
      inverse_scores.append(inverse_score)
      text[ind] = "o"


      continue

    else:
      text[ind] = antonym

  if len(negation_tokens) > 0:
    text_negless = [str(tok) for tok in text if tok not in [str(t) for t in negation_tokens]] #remove negation tokens from string if they're present
  else:
    text_negless = [str(tok) for tok in text] #if there are no negations present, just return every token from the text

  print(text_negless)

  ### Collect polarity scores for all words in list ###

  pol_scores = [return_polarity_scores(str(word)) for word in text_negless]


  #append inverse score for when antonyms aren't found
  if len(inverse_scores) > 0:
    for score in inverse_scores:
      pol_scores.append(inverse_score)


  # Return the mean score of the collected scores across all the words in the snippet
  score = round(sig(statistics.mean(pol_scores)), 2)
  label = polarity_label(score) #rond score af tot 2 decimalen na de komma

  print(score, label)
  return label

In [ ]:
sentiment_scorer("pride")

['pride']
0.66 4


4

In [ ]:
annos_en["sentiment_predictions"] = annos_en["text"].apply(sentiment_scorer)

In [ ]:
annos_en.head()

,source_file,sentence,begin,end,_annotation,text,_sentence_text,annotation,sentiment_predictions
5,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_167-303,185,193,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",moderate,"In the P.M. had a moderate breeze at East , wh...",3,3
16,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_1171-1330,1254,1258,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",fair,At Midnight the wind came to South-South-West ...,4,3
21,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_1480-1687,1542,1547,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",fresh,Cloudy weather ; Winds at South-West and South...,4,4
23,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_1480-1687,1555,1589,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",with which we made our Course good,Cloudy weather ; Winds at South-West and South...,4,3
24,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_1826-1944,1832,1844,"w.c.Emotions_Entities(ASP_SENT_NULL=SENTIMENT,...",steady brisk,Had a steady brisk Gale at South-South-West wi...,4,4


In [ ]:
annos_en["sentiment_predictions"] = pd.to_numeric(annos_en["sentiment_predictions"])
annos_en["annotation"] = pd.to_numeric(annos_en["annotation"])

In [ ]:
true = list(annos_en["annotation"])
pred = list(annos_en["sentiment_predictions"])

In [ ]:
print(classification_report(true, pred))
#system doesn't find any 1s and 5s

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        88
           2       0.49      0.19      0.28       366
           3       0.14      0.65      0.23       200
           4       0.69      0.34      0.46       686
           5       0.00      0.00      0.00        96

    accuracy                           0.30      1436
   macro avg       0.26      0.24      0.19      1436
weighted avg       0.47      0.30      0.32      1436



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
